# 🧪 Pytest Fundamentals

Master professional testing with pytest - from basics to advanced patterns used in production frameworks like Odibi.

---

## 📚 Key Terms Glossary

**Testing Types**:
- **Unit Test**: Test a single function/class in isolation (fast, focused)
- **Integration Test**: Test components working together (slower, broader)
- **E2E Test**: Test complete user workflow end-to-end (slowest, most realistic)

**Testing Concepts**:
- **Fixture**: Reusable test setup/teardown (like test data, temp files)
- **Mock**: Fake replacement for external dependencies (API, database, files)
- **Assertion**: Check that actual result matches expected (the actual test)
- **Parametrize**: Run same test with different inputs (data-driven testing)
- **Coverage**: Percentage of code executed by tests (line or branch level)
- **Marker**: Tag tests for selective execution (like `@pytest.mark.slow`)

**Common Issues**:
- **Regression**: When a fix or change breaks previously working code
- **Flaky Test**: Test that sometimes passes, sometimes fails (usually from timing, randomness, or external state)
- **Brittle Test**: Test that breaks easily when code changes, even if behavior is correct
- **Technical Debt**: Accumulated shortcuts that make code harder to maintain

## 🎯 Problem: Why Testing Matters for Frameworks

**Scenario**: You built a data pipeline framework. A user reports:
- "Pipeline crashes with Azure connections"
- "Transform fails on edge cases"
- "Config validation breaks with certain inputs"

**Without tests**:
- You manually test each scenario (hours)
- Fix breaks another feature (regression)
- Fear changing code (technical debt)

**With tests**:
- Run 416 tests in seconds
- Catch regressions immediately
- Refactor with confidence

**Odibi's testing stats** (416 tests total):
- Coverage: ~85%
- Test execution: ~5-10 seconds
- Prevents 90%+ of production bugs

## 🦉 First Principles: Testing Philosophy

### Test Pyramid

```
       /\      E2E Tests (few, slow, brittle)
      /  \     
     /____\    Integration Tests (moderate)
    /      \   
   /________\  Unit Tests (many, fast, focused)
```

**Unit Tests (80%)**:
- Test single function/class
- Fast (milliseconds)
- Example: `test_transform_adds_column()`

**Integration Tests (15%)**:
- Test components working together
- Slower (seconds)
- Example: `test_pipeline_reads_transforms_writes()`

**E2E Tests (5%)**:
- Test full user workflow
- Slowest (minutes)
- Example: `test_cli_generates_project()`

### Key Principles

1. **Fast**: Tests should run in seconds
2. **Isolated**: Each test independent
3. **Repeatable**: Same result every time
4. **Self-validating**: Pass/fail, no manual check
5. **Timely**: Written with code, not after

## ⚡ Part 1: Pytest Basics

### Installation

In [1]:
# Install pytest and plugins
!pip install pytest pytest-cov pytest-mock -q

### Simple Assertions

**Why this matters**: Pytest's `assert` statement gives detailed failure messages showing exactly what went wrong. Unlike `unittest` which requires `self.assertEqual()`, pytest uses Python's native `assert` with "assert rewriting" magic that provides rich diffs automatically.

**Key concepts**:
- `assert` - Basic check (pytest shows both sides of comparison)
- `pytest.approx()` - Floating-point comparisons (avoids precision issues)
- `pytest.raises()` - Test that code raises expected exceptions

In [2]:
# Create a simple module to test
def add(a, b):
    """Add two numbers."""
    return a + b

def divide(a, b):
    """Divide a by b."""
    if b == 0:
        raise ValueError("Cannot divide by zero")
    return a / b

class Calculator:
    def __init__(self, initial=0):
        self.value = initial
    
    def add(self, x):
        self.value += x
        return self.value
    
    def reset(self):
        self.value = 0

In [11]:
%%writefile test_basic.py
"""Basic pytest examples."""
import pytest

def add(a, b):
    return a + b

def divide(a, b):
    if b == 0:
        raise ValueError("Cannot divide by zero")
    return a / b

# Test functions must start with 'test_'
def test_add():
    assert add(2, 3) == 5
    assert add(-1, 1) == 0
    assert add(0, 0) == 0

def test_add_floats():
    """Floating-point precision requires approximate comparison."""
    result = add(0.1, 0.2)
    # Without pytest.approx, this would fail: 0.1 + 0.2 = 0.30000000000000004
    assert result == pytest.approx(0.3)  # Handle float precision

def test_divide():
    assert divide(10, 2) == 5
    assert divide(9, 3) == 3

def test_divide_by_zero():
    """Test exception handling.
    
    The 'match' parameter uses regex to verify the error message.
    This ensures we're catching the RIGHT error, not just any ValueError.
    """
    with pytest.raises(ValueError, match="Cannot divide by zero"):
        divide(10, 0)

Overwriting test_basic.py


In [12]:
# Run tests
!pytest test_basic.py -v

============================= test session starts =============================
platform win32 -- Python 3.11.4, pytest-7.4.0, pluggy-1.6.0 -- C:\Users\hodibi\AppData\Local\anaconda3\python.exe
cachedir: .pytest_cache
rootdir: c:\Users\hodibi\OneDrive - Ingredion\Desktop\Repos\Python-Mastery\foundations\02_pytest_fundamentals
plugins: anyio-3.5.0, dash-3.0.2, Faker-37.3.0, cov-7.0.0, mock-3.15.1
collecting ... collected 4 items

test_basic.py::test_add PASSED                                           [ 25%]
test_basic.py::test_add_floats PASSED                                    [ 50%]
test_basic.py::test_divide PASSED                                        [ 75%]
test_basic.py::test_divide_by_zero PASSED                                [100%]

============================== 4 passed in 0.35s ==============================


### Test Discovery

Pytest automatically finds tests matching:
- Files: `test_*.py` or `*_test.py`
- Functions: `test_*()`
- Classes: `Test*` (no `__init__`)
- Methods: `test_*()`

## ⚡ Part 2: Fixtures (Setup/Teardown)

**Why this matters**: Fixtures solve three critical testing problems:
1. **Code reuse** - Write setup once, use in many tests
2. **Test isolation** - Each test gets fresh data (no shared state bugs)
3. **Readability** - Test function shows what it tests, not setup noise

**Without fixtures (painful)**:
```python
def test_data_shape():
    df = pd.DataFrame({"id": [1,2,3], "value": [10,20,30]})  # Duplicate setup
    assert df.shape == (3, 2)

def test_data_columns():
    df = pd.DataFrame({"id": [1,2,3], "value": [10,20,30]})  # Same setup again!
    assert list(df.columns) == ["id", "value"]
```

**With fixtures (clean)**:
```python
@pytest.fixture
def sample_data():
    return pd.DataFrame({"id": [1,2,3], "value": [10,20,30]})

def test_data_shape(sample_data):  # Pytest auto-injects fixture
    assert sample_data.shape == (3, 2)

def test_data_columns(sample_data):  # Fresh copy for each test
    assert list(sample_data.columns) == ["id", "value"]
```

In [13]:
%%writefile test_fixtures.py
"""Fixture examples."""
import pytest
import pandas as pd
from pathlib import Path

# Simple fixture
@pytest.fixture
def sample_data():
    """Provide sample DataFrame."""
    return pd.DataFrame({
        "id": [1, 2, 3],
        "value": [10, 20, 30]
    })

def test_data_shape(sample_data):
    assert sample_data.shape == (3, 2)

def test_data_columns(sample_data):
    assert list(sample_data.columns) == ["id", "value"]

# Fixture with setup/teardown using yield
@pytest.fixture
def temp_csv(tmp_path):
    """Create temp CSV file with automatic cleanup.
    
    yield pattern:
    - Code BEFORE yield = setup (runs before test)
    - yield value = what test receives
    - Code AFTER yield = teardown (runs after test, even if test fails)
    """
    # Setup: create temp file
    csv_file = tmp_path / "data.csv"
    df = pd.DataFrame({"a": [1, 2], "b": [3, 4]})
    df.to_csv(csv_file, index=False)
    
    yield csv_file  # Provide to test
    
    # Teardown: cleanup (tmp_path auto-deletes, but this shows the pattern)
    print(f"Cleaned up {csv_file}")

def test_read_csv(temp_csv):
    df = pd.read_csv(temp_csv)
    assert len(df) == 2
    assert "a" in df.columns

# Fixture scopes control how often setup runs
# Scopes (fastest to slowest):
# - function (default): New instance per test function
# - class: Shared by all tests in a class
# - module: Shared by all tests in a file
# - session: Shared by entire test run (use for expensive DB connections, etc.)

@pytest.fixture(scope="module")
def expensive_resource():
    """Created ONCE per module (file), reused by all tests.
    
    Use case: Database connection, spark session, large file load.
    Trade-off: Faster, but tests share state (could leak bugs).
    """
    print("\nSetup expensive resource")
    resource = {"data": "expensive to create"}
    yield resource
    print("\nTeardown expensive resource")

def test_resource_1(expensive_resource):
    assert expensive_resource["data"] == "expensive to create"

def test_resource_2(expensive_resource):
    # Uses same instance as test_resource_1
    assert "data" in expensive_resource

Writing test_fixtures.py


In [14]:
!pytest test_fixtures.py -v -s

============================= test session starts =============================
platform win32 -- Python 3.11.4, pytest-7.4.0, pluggy-1.6.0 -- C:\Users\hodibi\AppData\Local\anaconda3\python.exe
cachedir: .pytest_cache
rootdir: c:\Users\hodibi\OneDrive - Ingredion\Desktop\Repos\Python-Mastery\foundations\02_pytest_fundamentals
plugins: anyio-3.5.0, dash-3.0.2, Faker-37.3.0, cov-7.0.0, mock-3.15.1
collecting ... collected 5 items

test_fixtures.py::test_data_shape PASSED
test_fixtures.py::test_data_columns PASSED
test_fixtures.py::test_read_csv PASSEDCleaned up C:\Users\hodibi\AppData\Local\Temp\pytest-of-hodibi\pytest-0\test_read_csv0\data.csv

test_fixtures.py::test_resource_1 
Setup expensive resource
PASSED
test_fixtures.py::test_resource_2 PASSED
Teardown expensive resource


============================= 5 passed in 12.89s ==============================


### Built-in Fixtures

Pytest provides powerful fixtures out of the box:

- **`tmp_path`**: Temporary directory (Path object) - Auto-deleted after test. Safe for parallel tests.
  - *Use case*: Testing file I/O without polluting file system
- **`tmp_path_factory`**: Create multiple temp dirs (session-scoped)
  - *Use case*: Shared temp directories across tests
- **`monkeypatch`**: Safely modify objects, dicts, environment variables
  - *Use case*: Mock environment vars, patch functions, fake system state
- **`capsys`**: Capture stdout/stderr
  - *Use case*: Test print statements, CLI output
- **`caplog`**: Capture log messages
  - *Use case*: Test logging behavior
- **`request`**: Access test context and metadata
  - *Use case*: Get test name, parametrize values in fixtures

## ⚡ Part 3: Parametrize (Data-Driven Tests)

**Why this matters**: Instead of writing loops or duplicate tests, parametrize runs the same test with different inputs. Each combination becomes a separate test case with its own pass/fail status.

**Why not loops?**
```python
# ❌ BAD: Loop stops at first failure, unclear which case failed
def test_palindrome_loop():
    for word in ["racecar", "madam", "hello"]:
        assert is_palindrome(word)  # "hello" fails - other cases not tested!
```

**Why parametrize?**
```python
# ✅ GOOD: Each case is a separate test, all run independently
@pytest.mark.parametrize("word", ["racecar", "madam", "hello"])
def test_palindrome(word):
    assert is_palindrome(word)  # Shows exactly which word failed
```

Benefits:
- All cases run (failure doesn't stop others)
- Clear which input failed
- Easier to add more test cases
- Better test reports

In [ ]:
%%writefile test_parametrize.py
"""Parametrize examples."""
import pytest

def is_palindrome(s):
    return s == s[::-1]

# Single parameter
@pytest.mark.parametrize("word", [
    "racecar",
    "madam",
    "noon",
])
def test_palindromes(word):
    assert is_palindrome(word)

# Multiple parameters
@pytest.mark.parametrize("input,expected", [
    ("racecar", True),
    ("hello", False),
    ("", True),
    ("a", True),
    ("ab", False),
])
def test_is_palindrome(input, expected):
    assert is_palindrome(input) == expected

# Named test IDs for clarity
@pytest.mark.parametrize("input,expected", [
    ("racecar", True),
    ("hello", False),
], ids=["palindrome", "not_palindrome"])
def test_with_ids(input, expected):
    assert is_palindrome(input) == expected

# Complex example from Odibi
@pytest.mark.parametrize("connection_type,format,expected_class", [
    ("local", "csv", "PandasEngine"),
    ("local", "parquet", "PandasEngine"),
    ("azure", "csv", "PandasEngine"),
])
def test_engine_selection(connection_type, format, expected_class):
    # Simulate engine selection logic
    engine = expected_class
    assert engine == expected_class

In [ ]:
!pytest test_parametrize.py -v

## ⚡ Part 4: Mocking (Isolate External Dependencies)

**Why mock?** Unit tests should be:
1. **Fast** - No waiting for networks, databases, or file I/O
2. **Deterministic** - Same result every time (no random failures)
3. **Isolated** - Test YOUR code, not external systems
4. **Safe** - Don't send real emails, charge credit cards, or delete files!

**What to mock**:
- ✅ External APIs (requests, httpx)
- ✅ Databases (SQLAlchemy, psycopg2)
- ✅ File system (os.path.exists, open)
- ✅ Time (time.time, datetime.now)
- ✅ Random (random.random)
- ✅ Environment variables (os.getenv)

**What NOT to mock**:
- ❌ Your own code (defeats the purpose of testing)
- ❌ Simple data structures (just use real ones)
- ❌ Pure functions (they're already deterministic)

**Mocking tools**:
- `monkeypatch` (pytest fixture) - Easiest for simple cases
- `unittest.mock` (stdlib) - More power, more complex
- `pytest-mock` (plugin) - Combines both (requires `pip install pytest-mock`)

In [ ]:
%%writefile test_mocking.py
"""Mocking examples."""
import pytest
from unittest.mock import Mock, MagicMock

# Function that calls external API
def fetch_user(user_id):
    import requests
    response = requests.get(f"https://api.example.com/users/{user_id}")
    return response.json()

# Test with monkeypatch
def test_fetch_user_monkeypatch(monkeypatch):
    """Mock requests without hitting real API."""
    def mock_get(url):
        mock_response = Mock()
        mock_response.json.return_value = {"id": 1, "name": "Alice"}
        return mock_response
    
    # Replace requests.get with mock
    import requests
    monkeypatch.setattr(requests, "get", mock_get)
    
    result = fetch_user(1)
    assert result["name"] == "Alice"

# Mock environment variables
def get_api_key():
    import os
    return os.getenv("API_KEY")

def test_api_key_env(monkeypatch):
    monkeypatch.setenv("API_KEY", "test-key-123")
    assert get_api_key() == "test-key-123"

# Mock file system
def test_file_exists(monkeypatch, tmp_path):
    """Mock Path.exists()."""
    from pathlib import Path
    
    def mock_exists(self):
        return True
    
    monkeypatch.setattr(Path, "exists", mock_exists)
    
    # Any path now "exists"
    assert Path("/fake/path").exists() == True

# Mock class method
class DataLoader:
    def load(self, path):
        # Expensive operation
        import pandas as pd
        return pd.read_csv(path)

def test_data_loader(monkeypatch):
    import pandas as pd
    
    def mock_load(self, path):
        return pd.DataFrame({"a": [1, 2]})
    
    monkeypatch.setattr(DataLoader, "load", mock_load)
    
    loader = DataLoader()
    df = loader.load("/fake/path.csv")
    assert len(df) == 2

In [ ]:
!pytest test_mocking.py -v

## ⚡ Part 5: Coverage (Measure Test Completeness)

In [ ]:
%%writefile math_utils.py
"""Math utilities to test coverage."""

def divide(a, b):
    if b == 0:
        raise ValueError("Cannot divide by zero")
    return a / b

def is_even(n):
    return n % 2 == 0

def process_list(items):
    if not items:
        return []
    
    result = []
    for item in items:
        if item > 0:
            result.append(item * 2)
        else:
            result.append(item)
    return result

In [ ]:
%%writefile test_math_utils.py
"""Tests for math_utils."""
import pytest
from math_utils import divide, is_even, process_list

def test_divide():
    assert divide(10, 2) == 5

def test_divide_zero():
    with pytest.raises(ValueError):
        divide(10, 0)

def test_is_even():
    assert is_even(4) == True
    assert is_even(3) == False

def test_process_list_empty():
    assert process_list([]) == []

def test_process_list_positive():
    assert process_list([1, 2, 3]) == [2, 4, 6]

# Missing: test with negative numbers!

In [ ]:
# Run with coverage report
!pytest test_math_utils.py --cov=math_utils --cov-report=term-missing

Coverage shows:
- Which lines executed
- Missing branches (if/else not tested)
- Percentage covered

**Target**: 80-90% coverage (100% not always practical)

## ⚡ Part 6: Markers and conftest.py

In [ ]:
%%writefile test_markers.py
"""Marker examples."""
import pytest
import sys

# Skip test
@pytest.mark.skip(reason="Not implemented yet")
def test_future_feature():
    assert False

# Conditional skip
@pytest.mark.skipif(sys.platform == "win32", reason="Fails on Windows")
def test_unix_only():
    assert True

# Expected to fail
@pytest.mark.xfail(reason="Known bug #123")
def test_known_bug():
    assert 1 == 2

# Custom markers (group tests)
@pytest.mark.slow
def test_slow_operation():
    import time
    time.sleep(0.1)
    assert True

@pytest.mark.integration
def test_database_integration():
    assert True

@pytest.mark.unit
def test_fast_unit():
    assert True

In [ ]:
# Run only specific markers
!pytest test_markers.py -v -m "unit"  # Only unit tests
!pytest test_markers.py -v -m "not slow"  # Skip slow tests

### conftest.py (Shared Fixtures)

Create `conftest.py` in test directory for shared fixtures:

In [ ]:
%%writefile conftest.py
"""Shared test fixtures and configuration."""
import pytest
import pandas as pd

@pytest.fixture
def sample_df():
    """Fixture available to all tests."""
    return pd.DataFrame({
        "id": [1, 2, 3],
        "value": [10, 20, 30]
    })

@pytest.fixture(scope="session")
def test_config():
    """Config created once per test session."""
    return {
        "env": "test",
        "debug": True
    }

# Register custom markers
def pytest_configure(config):
    config.addinivalue_line("markers", "slow: marks tests as slow")
    config.addinivalue_line("markers", "integration: integration tests")
    config.addinivalue_line("markers", "unit: unit tests")

## ⚡ Part 7: Testing Exceptions

In [ ]:
%%writefile test_exceptions.py
"""Exception testing examples."""
import pytest

def validate_age(age):
    if age < 0:
        raise ValueError("Age cannot be negative")
    if age > 150:
        raise ValueError("Age too high")
    return age

# Basic exception test
def test_negative_age():
    with pytest.raises(ValueError):
        validate_age(-1)

# Check exception message
def test_negative_age_message():
    with pytest.raises(ValueError, match="cannot be negative"):
        validate_age(-5)

# Capture exception for inspection
def test_exception_details():
    with pytest.raises(ValueError) as exc_info:
        validate_age(200)
    
    assert "too high" in str(exc_info.value)
    assert exc_info.type == ValueError

# Test multiple exceptions
@pytest.mark.parametrize("age,expected_msg", [
    (-1, "cannot be negative"),
    (200, "too high"),
])
def test_validation_errors(age, expected_msg):
    with pytest.raises(ValueError, match=expected_msg):
        validate_age(age)

In [ ]:
!pytest test_exceptions.py -v

## 🔍 Part 8: Odibi Testing Patterns Analysis

Let's analyze how Odibi's 416 tests are organized:

In [ ]:
# Odibi test structure
odibi_structure = """
tests/
├── unit/                    # 15 test files (unit tests)
│   ├── test_operations.py
│   ├── test_cli.py
│   ├── test_registry.py
│   └── ...
├── integration/             # 1 test file (integration tests)
│   └── test_cli_integration.py
├── fixtures/                # Test data
│   └── sample_data.csv
├── test_pipeline.py         # Core pipeline tests
├── test_config.py           # Configuration tests
├── test_connections_paths.py # Connection tests
└── conftest.py (likely)     # Shared fixtures
"""
print(odibi_structure)

### Key Patterns from Odibi Tests

**1. Class-based test organization** (from `test_pipeline.py`):

In [ ]:
%%writefile test_odibi_pattern1.py
"""Pattern 1: Class-based organization."""
import pytest
import tempfile
from pathlib import Path

class TestPipelineExecution:
    """Group related pipeline tests."""
    
    def setup_method(self):
        """Run before each test method."""
        self.test_dir = tempfile.mkdtemp()
        self.test_path = Path(self.test_dir)
        print(f"\nSetup: {self.test_dir}")
    
    def teardown_method(self):
        """Run after each test method."""
        import shutil
        shutil.rmtree(self.test_dir)
        print(f"\nTeardown: {self.test_dir}")
    
    def test_read_pipeline(self):
        assert self.test_path.exists()
    
    def test_write_pipeline(self):
        assert self.test_path.exists()

**2. Parametrize for connection types** (from `test_pandas_engine_full_coverage.py`):

In [ ]:
%%writefile test_odibi_pattern2.py
"""Pattern 2: Parametrize connections and formats."""
import pytest

@pytest.mark.parametrize("format,options", [
    ("csv", {"sep": ","}),
    ("parquet", {"engine": "pyarrow"}),
    ("json", {"orient": "records"}),
])
def test_read_formats(format, options):
    """Test reading different formats."""
    assert format in ["csv", "parquet", "json"]
    assert isinstance(options, dict)

**3. Monkeypatch for optional dependencies** (from `test_extras_imports.py`):

In [ ]:
%%writefile test_odibi_pattern3.py
"""Pattern 3: Test missing optional dependencies."""
import pytest
import sys

@pytest.mark.extras
def test_import_without_pyspark(monkeypatch):
    """Test graceful handling when PySpark not installed."""
    # Simulate missing pyspark
    monkeypatch.setitem(sys.modules, "pyspark", None)
    
    # Code should handle missing dependency
    try:
        import pyspark
        assert pyspark is None
    except ImportError:
        pass  # Expected

@pytest.mark.skipif("pyspark" not in sys.modules, reason="PySpark not installed")
def test_with_pyspark():
    """Only run if PySpark available."""
    pass

**4. Custom markers for test categories**:

In [ ]:
# In Odibi's tests:
# @pytest.mark.extras - Tests for optional features
# @pytest.mark.skip - Known issues or platform-specific

# Run only extra features tests:
# pytest -m extras

# Skip slow integration tests:
# pytest -m "not integration"

## 🏗️ Build: Create Test Suite for Mini Pipeline

Let's build a complete test suite for a mini data pipeline:

In [ ]:
%%writefile mini_pipeline.py
"""Mini data pipeline to test."""
import pandas as pd
from pathlib import Path
from typing import Callable

class Pipeline:
    def __init__(self, name: str):
        self.name = name
        self.transforms = []
    
    def add_transform(self, func: Callable):
        """Add transformation function."""
        self.transforms.append(func)
    
    def run(self, df: pd.DataFrame) -> pd.DataFrame:
        """Execute pipeline."""
        result = df.copy()
        for transform in self.transforms:
            result = transform(result)
        return result

def read_csv(path: Path) -> pd.DataFrame:
    """Read CSV file."""
    if not path.exists():
        raise FileNotFoundError(f"File not found: {path}")
    return pd.read_csv(path)

def write_csv(df: pd.DataFrame, path: Path):
    """Write CSV file."""
    df.to_csv(path, index=False)

# Sample transforms
def add_total_column(df: pd.DataFrame) -> pd.DataFrame:
    """Add total column."""
    if "quantity" in df.columns and "price" in df.columns:
        df["total"] = df["quantity"] * df["price"]
    return df

def filter_positive(df: pd.DataFrame) -> pd.DataFrame:
    """Filter positive values."""
    if "total" in df.columns:
        return df[df["total"] > 0]
    return df

In [ ]:
%%writefile test_mini_pipeline.py
"""Complete test suite for mini pipeline."""
import pytest
import pandas as pd
from pathlib import Path
from mini_pipeline import (
    Pipeline, read_csv, write_csv,
    add_total_column, filter_positive
)

# Fixtures
@pytest.fixture
def sample_df():
    """Sample DataFrame."""
    return pd.DataFrame({
        "quantity": [2, 3, 0],
        "price": [10, 20, 15]
    })

@pytest.fixture
def sample_csv(tmp_path, sample_df):
    """Sample CSV file."""
    csv_path = tmp_path / "data.csv"
    sample_df.to_csv(csv_path, index=False)
    return csv_path

# Unit tests
class TestPipeline:
    def test_create_pipeline(self):
        pipeline = Pipeline("test")
        assert pipeline.name == "test"
        assert len(pipeline.transforms) == 0
    
    def test_add_transform(self):
        pipeline = Pipeline("test")
        pipeline.add_transform(add_total_column)
        assert len(pipeline.transforms) == 1
    
    def test_run_empty_pipeline(self, sample_df):
        pipeline = Pipeline("test")
        result = pipeline.run(sample_df)
        pd.testing.assert_frame_equal(result, sample_df)
    
    def test_run_with_transform(self, sample_df):
        pipeline = Pipeline("test")
        pipeline.add_transform(add_total_column)
        result = pipeline.run(sample_df)
        assert "total" in result.columns

# I/O tests
class TestIO:
    def test_read_csv(self, sample_csv):
        df = read_csv(sample_csv)
        assert len(df) == 3
        assert "quantity" in df.columns
    
    def test_read_nonexistent_file(self, tmp_path):
        with pytest.raises(FileNotFoundError, match="File not found"):
            read_csv(tmp_path / "missing.csv")
    
    def test_write_csv(self, tmp_path, sample_df):
        output_path = tmp_path / "output.csv"
        write_csv(sample_df, output_path)
        assert output_path.exists()
        
        # Verify contents
        df = pd.read_csv(output_path)
        pd.testing.assert_frame_equal(df, sample_df)

# Transform tests
class TestTransforms:
    def test_add_total_column(self, sample_df):
        result = add_total_column(sample_df)
        assert "total" in result.columns
        assert list(result["total"]) == [20, 60, 0]
    
    def test_add_total_missing_columns(self):
        df = pd.DataFrame({"other": [1, 2]})
        result = add_total_column(df)
        assert "total" not in result.columns
    
    def test_filter_positive(self):
        df = pd.DataFrame({"total": [10, -5, 0, 20]})
        result = filter_positive(df)
        assert len(result) == 2
        assert list(result["total"]) == [10, 20]

# Integration test
@pytest.mark.integration
def test_full_pipeline_integration(tmp_path):
    """Test complete pipeline workflow."""
    # Setup input data
    input_path = tmp_path / "input.csv"
    df = pd.DataFrame({
        "quantity": [2, 3, -1],
        "price": [10, 20, 15]
    })
    write_csv(df, input_path)
    
    # Create pipeline
    pipeline = Pipeline("full_test")
    pipeline.add_transform(add_total_column)
    pipeline.add_transform(filter_positive)
    
    # Execute
    input_df = read_csv(input_path)
    result = pipeline.run(input_df)
    
    # Verify
    assert len(result) == 2  # Filtered negative
    assert "total" in result.columns
    
    # Write output
    output_path = tmp_path / "output.csv"
    write_csv(result, output_path)
    assert output_path.exists()

# Parametrized tests
@pytest.mark.parametrize("quantity,price,expected", [
    (2, 10, 20),
    (3, 20, 60),
    (0, 15, 0),
    (-1, 10, -10),
])
def test_total_calculation(quantity, price, expected):
    df = pd.DataFrame({"quantity": [quantity], "price": [price]})
    result = add_total_column(df)
    assert result["total"].iloc[0] == expected

## ✅ Validate: Run Tests and Check Coverage

In [ ]:
# Run all tests with verbose output
!pytest test_mini_pipeline.py -v

In [ ]:
# Run with coverage
!pytest test_mini_pipeline.py --cov=mini_pipeline --cov-report=term-missing

In [ ]:
# Run only unit tests
!pytest test_mini_pipeline.py -v -m "not integration"

In [ ]:
# Generate HTML coverage report
!pytest test_mini_pipeline.py --cov=mini_pipeline --cov-report=html
print("\nOpen htmlcov/index.html to see detailed coverage")

## 🎯 Summary: Pytest Mastery Checklist

### You've learned:

✅ **Basics**: Assertions, test discovery, running tests

✅ **Fixtures**: Setup/teardown, scope, `tmp_path`, shared fixtures

✅ **Parametrize**: Data-driven tests, multiple inputs

✅ **Mocking**: `monkeypatch`, isolate dependencies

✅ **Coverage**: Measure completeness, find gaps

✅ **Markers**: Skip, xfail, custom markers, organize tests

✅ **Exceptions**: Test error handling

✅ **Odibi patterns**: Class-based tests, fixtures, real-world examples

### Key Commands:

```bash
# Run all tests
pytest

# Verbose output
pytest -v

# Run specific file
pytest test_file.py

# Run specific test
pytest test_file.py::test_name

# Run with markers
pytest -m "unit"
pytest -m "not slow"

# Coverage
pytest --cov=module --cov-report=term-missing
pytest --cov=module --cov-report=html

# Show print statements
pytest -s

# Stop on first failure
pytest -x

# Run last failed tests
pytest --lf
```

### Next Steps:

1. Complete exercises in `exercises.ipynb`
2. Read Odibi test analysis in `odibi_test_analysis.md`
3. Practice: Write tests for your own code
4. Explore advanced topics: `pytest-asyncio`, `pytest-benchmark`, `hypothesis`